In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('MSFT.csv')
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,%K,%R,RSI
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.059946,1031788800,NaN,NaN,NaN
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062087,308160000,NaN,NaN,NaN
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063158,133171200,NaN,NaN,NaN
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.061552,67766400,NaN,NaN,NaN
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.060482,47894400,NaN,NaN,NaN
5,1986-03-20,0.098090,0.098090,0.094618,0.095486,0.058876,58435200,NaN,NaN,NaN
6,1986-03-21,0.095486,0.097222,0.091146,0.092882,0.057270,59990400,NaN,NaN,NaN
7,1986-03-24,0.092882,0.092882,0.089410,0.090278,0.055665,65289600,NaN,NaN,NaN
8,1986-03-25,0.090278,0.092014,0.089410,0.092014,0.056735,32083200,14.537005,NaN,NaN
9,1986-03-26,0.092014,0.095486,0.091146,0.094618,0.058341,22752000,29.775491,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9712 entries, 0 to 9711
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       9712 non-null   object 
 1   Open       9712 non-null   float64
 2   High       9712 non-null   float64
 3   Low        9712 non-null   float64
 4   Close      9712 non-null   float64
 5   Adj Close  9712 non-null   float64
 6   Volume     9712 non-null   int64  
 7   %K         9704 non-null   float64
 8   %R         9699 non-null   float64
 9   RSI        9698 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 758.9+ KB


In [4]:
del data['Date']

In [5]:
data = data.dropna(axis=0)

In [6]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume,%K,%R,RSI
14,0.096354,0.098958,0.096354,0.096354,0.059411,23040000,65.476190,-50.003594,48.387216
15,0.096354,0.097222,0.096354,0.096354,0.059411,26582400,57.142857,-50.003594,48.387216
16,0.096354,0.097222,0.092882,0.094618,0.058341,16560000,42.857143,-62.502696,45.018991
17,0.094618,0.097222,0.094618,0.095486,0.058876,10252800,40.476190,-46.153846,47.005348
18,0.095486,0.098090,0.095486,0.097222,0.059946,12153600,47.619048,-18.181818,50.831369
19,0.097222,0.098958,0.095486,0.098090,0.060482,13881600,66.666667,-9.090909,52.671272
20,0.098958,0.101563,0.098958,0.099826,0.061552,17222400,79.044552,-14.292756,56.201333
21,0.099826,0.101563,0.099826,0.100694,0.062087,12153600,84.397502,-7.150485,57.892445
22,0.100694,0.100694,0.097222,0.100694,0.062087,9302400,84.392808,-8.342119,57.892445
23,0.100694,0.105035,0.099826,0.104167,0.064229,31910400,88.032556,-7.142270,64.290516


In [7]:
train_data = pd.DataFrame(data.iloc[0:int(len(data)*0.8), :])
test_data = pd.DataFrame(data.iloc[int(len(data)*0.8):len(data), :])

In [8]:
print(len(train_data))
print(len(test_data))

7758
1940


In [9]:
train_data.head()

,Open,High,Low,Close,Adj Close,Volume,%K,%R,RSI
14,0.096354,0.098958,0.096354,0.096354,0.059411,23040000,65.476190,-50.003594,48.387216
15,0.096354,0.097222,0.096354,0.096354,0.059411,26582400,57.142857,-50.003594,48.387216
16,0.096354,0.097222,0.092882,0.094618,0.058341,16560000,42.857143,-62.502696,45.018991
17,0.094618,0.097222,0.094618,0.095486,0.058876,10252800,40.476190,-46.153846,47.005348
18,0.095486,0.098090,0.095486,0.097222,0.059946,12153600,47.619048,-18.181818,50.831369


In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [15]:
import numpy as np

x_train = []
y_train = [] 

x_test = []
y_test = []

for i in range(100, scaled_train_data.shape[0]):
    x_train.append(scaled_train_data[i-100: i, :])
    y_train.append(scaled_train_data[i, 3])

for i in range(100, scaled_test_data.shape[0]):
    x_test.append(scaled_test_data[i-100: i, :])
    y_test.append(scaled_test_data[i, 3])

x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)

In [16]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(7658, 100, 9)
(7658,)
(1840, 100, 9)
(1840,)


In [34]:
# WORK IN PROGRESS
from keras.layers import Dropout, LSTM, Dense, Input
from keras.models import Sequential
from keras.regularizers import l2

model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2],)))
model.add(LSTM(units=30, activation='relu'))
model.add(Dropout(0.2))
# model.add(LSTM(units = 40, activation = 'relu'))
# model.add(Dropout(0.3))
# model.add(LSTM(units = 80, activation = 'sigmoid'))
# model.add(Dropout(0.4))
# model.add(LSTM(units = 160, activation = 'relu'))
# model.add(Dropout(0.5))
model.add(Dense(units = 1))


In [35]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_11 (LSTM)                  │ (None, 30)             │         4,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,831 (18.87 KB)

 Trainable params: 4,831 (18.87 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.compile(optimizer = 'adam', loss = 'mse', metrics=['mae'])
model.fit(x_train, y_train, epochs = 80)

Epoch 1/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0253 - mae: 0.1049
Epoch 2/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0039 - mae: 0.0434
Epoch 3/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0031 - mae: 0.0375
Epoch 4/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0025 - mae: 0.0348
Epoch 5/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0024 - mae: 0.0336
Epoch 6/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0021 - mae: 0.0326
Epoch 7/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0020 - mae: 0.0316
Epoch 8/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0018 - mae: 0.0305
Epoch 9/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0017 - mae: 0.0306
Epoch 10/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0017 - mae: 0.0294
Epoch 11/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0015 - mae: 0.0279
Epoch 12/80
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0016 - mae: 0.0285
Epoch 13/80
240/240 ━━━━━

In [37]:
score = model.evaluate(x_test, y_test)

58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0057 - mae: 0.0624
